In [ ]:
#default_exp training.tuner

# Tuning API
> Modules for task fine-tuning

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.all import *
from fastai.metrics import *
from fastai.losses import *
from fastai.optimizer import *
# NOTE: Placeholder imports, remove once we are ready for release

## Integration with [fastai](https://docs.fast.ai)

Since `fastai` is a _very_ lightweight framework that is easily approachable and incorporates state-of-the-art ideas, `AdaptNLP` bridges the gap between HuggingFace and fastai, allowing you to train with their framework through the `AdaptiveLearner` and `*Tuner` classes

In [ ]:
#export
class AdaptiveLearner(Learner):
    """
    A base fastai `Learner` that overrides `_split` and `_do_one_batch` to
    have it work with HuggingFace datasets and models
    """
    def _split(self, b):
        "Assign `self.xb` to model input and labels"
        self.xb = b
        if 'labels' in b.keys(): self.yb = b['labels'].unsqueeze(0)
    
    def _do_one_batch(self):
        "Move a batch of data to a device, get predictions, calculate the loss, and perform backward pass"
        self.xb = {k:v.to(self.device) for k,v in self.xb.items()} # See if `to_device` fixes this
        self.yb = self.yb.to(self.device)
        out = self.model(**self.xb)
        self.pred = out['logits'].to(self.device)
        self('after_pred')
        self.loss_grad = out['loss'].to(self.device)
        self.loss = self.loss_grad.clone()
        self('after_loss')
        if not self.training or not len(self.yb): return
        self('before_backward')
        self.loss_grad.backward()
        self._with_events(self.opt.step, 'step', CancelStepException)
        self.opt.zero_grad()
    
    def predict(self, inp): raise NotImplementedError()

The `AdaptiveLearner` class is the base class you should use when fine-tuning on `HuggingFace` models and datasets. It assumes that the dataset will *always* be a dictionary, with labels always being a `labels` key. Futher task-specific `Learner`'s are detailed below:

In [ ]:
#export
from transformers import AutoModelForSequenceClassification

In [ ]:
#export
class SequenceClassificationTuner(AdaptiveLearner):
    """
    A `AdaptiveLearner` with good defaults for Sequence Classification tasks
    
    **Valid kwargs and defaults:**
      - `lr`:float = 0.001
      - `splitter`:function = `trainable_params`
      - `cbs`:list = None
      - `path`:Path = None
      - `model_dir`:Path = 'models'
      - `wd`:float = None
      - `wd_bn_bias`:bool = False
      - `train_bn`:bool = True
      - `moms`: tuple(float) = (0.95, 0.85, 0.95)
    
    """
    def __init__(
        self,
        dls:DataLoaders, # A set of DataLoaders
        model, # A HuggingFace model
        loss_func = CrossEntropyLossFlat(), # A loss function
        metrics = [accuracy, F1Score()], # Metrics to monitor the training with
        opt_func = Adam, # A fastai or torch Optimizer
        additional_cbs = None, # Additional Callbacks to have always tied to the Tuner
        **kwargs, # kwargs for `Learner.__init__`
    ):
        additional_cbs = listify(additional_cbs)
        for arg in 'dls,model,loss_func,metrics,opt_func,cbs'.split(','): 
            if arg in kwargs.keys(): kwargs.pop(arg) # Pop all existing kwargs
        model = AutoModelForSequenceClassification.from_pretrained(model, num_labels=len(dls[0].categorize.classes))
        super().__init__(
            dls, 
            model, 
            loss_func = loss_func, 
            metrics = metrics, 
            opt_func = opt_func, 
            cbs=additional_cbs, 
            **kwargs
        )